# Building Machine Learning Classifiers: Model selection

### Read in & clean text

In [6]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

#Feature creation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

In [7]:
#Natural Language Processing
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Split into train/test

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize text

In [9]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,7163,7164,7165,7166,7167,7168,7169,7170,7171,7172
0,31,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31,12.9,0.306597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40,20.0,0.252710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,133,6.0,0.137522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31,3.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Final evaluation of models

In [10]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [12]:
#Try random forest grid search
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train_vect, y_train)
    y_pred = rf_model.predict(X_test_vect)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))
    
    
for n_est in [10, 50, 100, 150]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.248 / Accuracy: 0.899
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.544 / Accuracy: 0.939
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.678 / Accuracy: 0.957
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.732 / Accuracy: 0.964
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.275 / Accuracy: 0.903
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.557 / Accuracy: 0.941
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.698 / Accuracy: 0.96
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.799 / Accuracy: 0.973
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.201 / Accuracy: 0.893
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.557 / Accuracy: 0.941
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.658 / Accuracy: 0.954
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.792 / Accuracy: 0.972
Est: 150 / Depth: 10 ---- Precision: 1.0 / Recall: 0.282 / Accuracy: 0.904
Est: 150 / Depth: 20 ---- Pr

In [13]:
#Take this parameter combination: 
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 1.609 / Predict time: 0.057 ---- Precision: 1.0 / Recall: 0.785 / Accuracy: 0.971


In [15]:
#Try Gradient Boosting
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train_vect, y_train)
    y_pred = gb_model.predict(X_test_vect)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.867
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.919 / Recall: 0.685 / Accuracy: 0.95
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.881 / Recall: 0.792 / Accuracy: 0.958
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.867
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.927 / Recall: 0.772 / Accuracy: 0.961
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.908 / Recall: 0.799 / Accuracy: 0.962
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.867
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.934 / Recall: 0.758 / Accuracy: 0.961
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.925 / Recall: 0.826 / Accuracy: 0.968


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.866
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.934 / Recall: 0.758 / Accuracy: 0.961
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.931 / Recall: 0.812 / Accuracy: 0.967
Est: 100 / Depth: 3 / LR: 0.01 ---- Precision: 0.926 / Recall: 0.503 / Accuracy: 0.928
Est: 100 / Depth: 3 / LR: 0.1 ---- Precision: 0.933 / Recall: 0.752 / Accuracy: 0.96
Est: 100 / Depth: 3 / LR: 1 ---- Precision: 0.879 / Recall: 0.779 / Accuracy: 0.956
Est: 100 / Depth: 7 / LR: 0.01 ---- Precision: 0.919 / Recall: 0.611 / Accuracy: 0.941
Est: 100 / Depth: 7 / LR: 0.1 ---- Precision: 0.935 / Recall: 0.779 / Accuracy: 0.963
Est: 100 / Depth: 7 / LR: 1 ---- Precision: 0.896 / Recall: 0.805 / Accuracy: 0.961
Est: 100 / Depth: 11 / LR: 0.01 ---- Precision: 0.945 / Recall: 0.691 / Accuracy: 0.953
Est: 100 / Depth: 11 / LR: 0.1 ---- Precision: 0.929 / Recall: 0.785 / Accuracy: 0.963
Est: 100 / Depth: 11 / LR: 1 ---- Precision: 0.944 / Recall

In [16]:
#Take this combination of parameters
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 107.071 / Predict time: 0.085 ---- Precision: 0.938 / Recall: 0.805 / Accuracy: 0.967


In [ ]:
#Gradient Boosting seems to perform better in terms of